In [1]:
import polars_ds
import polars as pl
import numpy as np 

# Num Extensions

In [2]:
size = 100_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=100_000)).astype(np.int32)
    , "predicted": np.random.random(size=100_000)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,"""a""",0.457934,0.6423,0,100000,-100000,1,0.890699,"""a"""
0.841471,"""a""",0.459135,0.735028,1,100001,-99999,0,0.388504,"""a"""
0.909297,"""a""",0.307611,0.634786,2,100002,-99998,1,0.642528,"""a"""
0.14112,"""a""",0.95301,0.074787,3,100003,-99997,0,0.327906,"""a"""
-0.756802,"""a""",0.472305,0.905882,4,100004,-99996,1,0.227964,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pl.col("x1").num.jaccard(pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT
df.select(
    pl.col("f").num.rfft()
).head()

f
list[f64]
"[1.812028, 0.0]"
"[1.812028, -0.000002]"
"[1.812028, -0.000005]"
"[1.812028, -0.000007]"
"[1.812028, -0.00001]"


In [5]:
# Least Square (Linear Regression)
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)

y
list[f64]
"[2.0, -1.0]"


In [6]:
df.lazy().select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
).collect()

y
list[f64]
"[2.0, -1.0]"


In [7]:
df.select(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False).over(pl.col("dummy"))
).head()

list_float
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [8]:
df.group_by("dummy").agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)


dummy,list_float
str,list[f64]
"""a""","[2.0, -1.0]"
"""b""","[2.0, -1.0]"


In [9]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pl.col("y").num.cond_entropy(pl.col("x1"))
)

y
f64
-0.0


In [10]:
df.group_by("dummy_groups").agg(
    pl.col("actual").num.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").num.bce(pl.col("predicted")).alias("log loss"),
    pl.col("actual").num.binary_metrics_combo(pl.col("predicted")).alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""a""",0.331461,0.994676,0.504398,0.503277,0.251918,0.506909,0.503755
"""b""",0.332576,0.999094,0.500683,0.498081,0.249689,0.500449,0.501698


# Str Extension

In [11]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [12]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""hello"""
"""church"""
"""world"""
"""going"""
"""to"""


In [13]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
"""hello"""
"""world"""
"""church"""
"""go"""


In [14]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [15]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [16]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [17]:
df2.filter(
    pl.col("word").str2.levenshtein("world") == 1
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [18]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [19]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "leven", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [20]:

df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [21]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-1.497465
-0.859263
-0.596512


In [22]:
# Genenrate random sample, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
)

a,random
f64,f64
null,null
null,null
-1.497465,0.263812
-0.859263,-0.600834
-0.596512,0.085847
-0.797408,1.611438
0.065377,0.711696
0.486381,-0.431943
0.657458,1.799759


In [23]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
)

a,random_str
f64,str
null,null
null,null
-1.497465,"""n2"""
-0.859263,"""B"""
-0.596512,"""y1oL"""
-0.797408,"""KK0"""
0.065377,"""gM5"""
0.486381,"""4pBH"""
0.657458,"""r"""


In [24]:
# Genenrate fixed size random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
)

a,random_str
f64,str
null,null
null,null
-1.497465,"""TlshF"""
-0.859263,"""vrsiJ"""
-0.596512,"""tXvOF"""
-0.797408,"""XsFtP"""
0.065377,"""lN3oB"""
0.486381,"""yYIhn"""
0.657458,"""pXyXn"""


In [25]:
df.with_columns(
    # Sample from normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.sample_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.sample_uniform(low = 0., high = None, respect_null=True).alias("test2")
).head()

a,test1,test2
f64,f64,f64
null,2.015709,null
null,0.100036,null
-1.497465,0.754484,2.671293
-0.859263,-0.107083,0.509611
-0.596512,1.619745,1.599823


In [26]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pl.col("a").stats.sample_normal(mean = 0.5, std = 1.).alias("test1")
    , pl.col("a").stats.sample_normal(mean = 0.5, std = 2.).alias("test2")
).select(
    pl.col("test1").stats.ttest_ind(pl.col("test2"), equal_var = False).alias("t-test")
    , pl.col("test1").stats.normal_test().alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.241144,0.809478,0.162122,0.922138


In [27]:
size = 5000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.442217,0.026176,1,9
1,0.818966,0.583352,3,3
2,0.41901,0.616824,0,3
0,0.006135,0.754259,1,2
1,0.781227,0.799708,2,7


In [28]:
# In dataframe statistical tests!
print(df.select(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
))

shape: (1, 3)
┌─────────────────────┬────────────────────┬───────────────────┐
│ t-test              ┆ chi2-test          ┆ f-test            │
│ ---                 ┆ ---                ┆ ---               │
│ struct[2]           ┆ struct[2]          ┆ struct[2]         │
╞═════════════════════╪════════════════════╪═══════════════════╡
│ {2.115272,0.034431} ┆ {32.5811,0.631989} ┆ {1.881212,0.1108} │
└─────────────────────┴────────────────────┴───────────────────┘


In [29]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = False).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),-
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{1.598889,0.10994}","{36.896559,0.427279}","{-1.900136,-0.1079}"
1,"{1.806081,0.070996}","{44.404524,0.158728}","{-0.346406,-0.846645}"
2,"{0.257484,0.796821}","{26.412285,0.878774}","{-0.350121,-0.844067}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [30]:
import polars_ds as pld
df = pl.DataFrame({
    "id": range(1000),
    "val1": np.random.random(size=1000), 
    "val2": np.random.random(size=1000), 
    "val3": np.random.random(size=1000),
    "r": np.random.random(size=1000),
    "rh": np.random.random(size=1000)*10,
})

In [31]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.select(
    pld.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "inf", # L Infinity distance 
        parallel = True 
    )
).head()

""
u32
11
3
8
11
9


In [32]:
df.select(
    pld.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        dist = "l1", # L 1 distance 
        parallel = True 
    )
).head()

r
u32
2
65
18
105
1


In [33]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pl.col("id").num.knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends")
).head()

id,val1,val2,val3,r,rh,best friends
i64,f64,f64,f64,f64,f64,list[u64]
0,0.946279,0.738426,0.362952,0.421689,1.876017,"[0, 958, … 313]"
1,0.741668,0.981829,0.883407,0.533584,6.247939,"[1, 460, … 906]"
2,0.758454,0.726879,0.96088,0.110808,4.053351,"[2, 568, … 834]"
3,0.96918,0.571118,0.91598,0.474751,4.305541,"[3, 641, … 82]"
4,0.432479,0.708105,0.345469,0.349741,1.646762,"[4, 379, … 389]"


In [34]:
# Filter to only points near the given point
df.filter(
    pld.query_radius(
        [0.5, 0.5, 0.5],
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        radius = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
4,0.432479,0.708105,0.345469,0.349741,1.646762
6,0.617363,0.456153,0.625416,0.603103,4.263803
7,0.329858,0.303322,0.728363,0.853283,9.724125
15,0.607162,0.131546,0.57634,0.986423,3.408407
16,0.216038,0.581832,0.682663,0.169455,9.051951


In [35]:
# Haversine distance is available when dimension is 2
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        radius = 10, # in km
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
103,0.427545,0.537531,0.001032,0.252425,0.520078
135,0.505411,0.475068,0.63048,0.308946,7.205082
245,0.506767,0.557298,0.850589,0.949393,7.133072
261,0.534525,0.475023,0.691376,0.212086,8.101366
275,0.46973,0.526947,0.404464,0.120119,9.040323


In [36]:
df.filter(
    pld.query_radius(
        [0.5, 0.5],
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        # radius can also be an existing column in the dataframe.
        radius = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
i64,f64,f64,f64,f64,f64
135,0.505411,0.475068,0.63048,0.308946,7.205082
245,0.506767,0.557298,0.850589,0.949393,7.133072
261,0.534525,0.475023,0.691376,0.212086,8.101366
275,0.46973,0.526947,0.404464,0.120119,9.040323
323,0.549024,0.453889,0.808333,0.470312,7.627118
